In [137]:
import pandas as pd
import numpy as np

X_train = np.array(pd.read_csv('./data/mecab/X_train.csv'))
X_test = np.array(pd.read_csv('./data/mecab/X_test.csv'))
y_train = pd.read_csv('./data/mecab/y_train.csv')
y_test =  pd.read_csv('./data/mecab/y_test.csv')

In [138]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

739999 185002 739999 185002


In [123]:
from keras.utils import np_utils

In [124]:
y_train.astype('str')
y_test.astype('str')

,0,1,2
0,H,49,493
1,F,42,422
2,I,56,562
3,I,56,562
4,C,10,101
...,...,...,...
184997,N,75,752
184998,F,42,421
184999,C,10,103
185000,G,47,474


In [140]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train['0'] = le.fit_transform(y_train['0'])
y_test['0'] = le.fit_transform(y_test['0'])

y_train['1'] = le.fit_transform(y_train['1'])
y_test['1'] = le.fit_transform(y_test['1'])

y_train['2'] = le.fit_transform(y_train['2'])
y_test['2'] = le.fit_transform(y_test['2'])

In [126]:
y_train['0'] = np_utils.to_categorical(y_train['0'])
y_test['0'] = np_utils.to_categorical(y_test['0'])

y_train['1'] = np_utils.to_categorical(y_train['1'])
y_test['1'] = np_utils.to_categorical(y_test['1'])

y_train['2'] = np_utils.to_categorical(y_train['2'])
y_test['2'] = np_utils.to_categorical(y_test['2'])

In [58]:
y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [136]:
y_train

,0,1,2
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0
...,...,...,...
739994,0.0,0.0,0.0
739995,0.0,0.0,0.0
739996,0.0,0.0,0.0
739997,0.0,0.0,0.0


In [142]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

vocab_size = 15778
embedding_dim = 200
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(3, activation='softmax'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=10, callbacks=[es, mc], batch_size=128, validation_split=0.2)

Epoch 1/10
4625/4625 [==============================] - 150s 32ms/step - loss: 613.8893 - acc: 0.9994 - val_loss: 618.7701 - val_acc: 0.9996

Epoch 00001: val_acc improved from -inf to 0.99964, saving model to best_model.h5
Epoch 2/10
2336/4625 [==============>...............] - ETA: 1:11 - loss: 618.7925 - acc: 0.9996

KeyboardInterrupt: 

In [143]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

5782/5782 [==============================] - 18s 3ms/step - loss: 607.7070 - acc: 0.9995

 테스트 정확도: 0.9995


In [144]:
import pandas as pd
import numpy as np

import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from eunjeon import Mecab
from tqdm import tqdm # 진행상황 Progress Bar를 위한 tqdm library
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
mecab = Mecab()

In [ ]:
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = mecab.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))